In [ ]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import random

# Function to convert a single text file to CSV with headers and set the "Activity" value
def txt_to_csv(input_file, output_file):
    activity_mapping = {
        "Bowing": 1,
        "Clapping": 1, 
        "Handshaking": 1,
        "Hugging": 1,
        "Jumping": 1,
        "Running": 1,
        "Seating": 1,
        "Standing": 1,
        "Walking": 1,
        "Waving": 1
    }
    default_activity = 0

    headers = ["R-Bic", "R-Tri", "L-Bic", "L-Tri", "R-Thi", "R-Ham", "L-Thi", "L-Ham", "Activity"]

    file_name = os.path.splitext(os.path.basename(input_file))[0]
    activity = activity_mapping.get(file_name, default_activity)

    with open(input_file, 'r') as txt_file, open(output_file, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(headers)
        for line in txt_file:
            row = line.strip().split('\t')
            row.append(activity)
            csv_writer.writerow(row)

# Function to convert .txt files to .csv with headers and set the "Activity" value
def convert_txt_to_csv_in_subfolders(root_directory, output_stft_directory, sample_and_create_stft_func):
    for subfolder in os.listdir(root_directory):
        subfolder_path = os.path.join(root_directory, subfolder)
        if os.path.isdir(subfolder_path):
            # Check if the subfolder is a subject folder
            if subfolder.startswith('sub'):  # Adjust the condition based on your subject folder naming convention
                subject_number = subfolder.split('sub')[1]  # Extract subject number from folder name
                for filename in os.listdir(subfolder_path):
                    if filename.endswith('.txt'):
                        txt_file_path = os.path.join(subfolder_path, filename)
                        csv_file_path = os.path.splitext(txt_file_path)[0] + '.csv'
                        txt_to_csv(txt_file_path, csv_file_path)
                        print(f"Converted: {txt_file_path} to {csv_file_path}")
                        
                        # Sampling and STFT creation
                        sample_and_create_stft_func(csv_file_path, output_stft_directory, subject_number)

# Function to sample CSV data randomly and create Short-Time Fourier Transform (STFT)
def sample_and_create_stft_random(csv_file_path, output_directory, subject_number):
    df = pd.read_csv(csv_file_path)
    activity = df['Activity'].iloc[0]
    sensor_columns = ["R-Bic", "R-Tri", "L-Bic", "L-Tri", "R-Thi", "R-Ham", "L-Thi", "L-Ham"]

    sample_size = 1000
    num_samples = df.shape[0] // sample_size
    num_segments_per_channel = 15
    file_name = os.path.splitext(os.path.basename(csv_file_path))[0]

    for sensor in sensor_columns:
        for i in range(num_segments_per_channel):
            # Randomly select a start index within the valid range
            start_index = random.randint(0, num_samples - 1) * sample_size
            end_index = start_index + sample_size
            segment = df.loc[start_index:end_index - 1, sensor].values

            f, t, Zxx = signal.stft(segment, fs=1.0)  # Adjust nperseg as needed
            
            # Plot STFT without labels
            plt.figure(figsize=(2.33333, 2.33333))
            plt.pcolormesh(t, f, np.abs(Zxx))  # Magnitude of STFT
            plt.axis('off')  # Turn off axis labels and ticks
            plt.gca().set_aspect('auto')  # Adjust aspect ratio for better visualization
            
            # Save STFT images in time-frequency format in activity-wise subfolders
            activity_output_directory = os.path.join(output_directory, f"Activity_{activity}")
            os.makedirs(activity_output_directory, exist_ok=True)
            
            stft_image_path = os.path.join(activity_output_directory, f"STFT_Sub{subject_number}{file_name}_sensor{sensor}_random{i + 1}.png")
            plt.savefig(stft_image_path, bbox_inches='tight', pad_inches=0, transparent=True)
            plt.close()
            print(f"Saved STFT image: {stft_image_path}")

# Specify the root directory containing the subfolders and the directory to save STFT images
root_directory = 'D:/EMG Physical Action Data Set'
output_stft_directory = 'C:/Users/saisr/OneDrive/Desktop/stft_images_output'  # Change this path

# Call the function to convert .txt files to .csv with headers and set the "Activity" value
convert_txt_to_csv_in_subfolders(root_directory, output_stft_directory, sample_and_create_stft_random)